# Count the signs

* ~~Write out the video output file. - this may need to be done inside the yolo file.~~
* Analyse the file, to get some idea of approach to counting.
* Extend the model so that it has an extra dictionary for the entry for the output so that it can overwrite the answer it provides.
* Then write out the end file so that 

In [1]:
#read the file in so that it pushes each found object into a dictionary with the frame as the key.
import os
base_dir = '/Users/datascience4/documents/yolo3/keras-yolo3/output/'

file = 'B4087_NB_RAV_R07_180519074106_completed_counting.txt'

file = open(os.path.join(base_dir, file), 'r')
file_read = file.readlines()
file.close()

In [2]:
# print(file_read)

In [3]:
import re
dic = {}
#leave the frame outside.
frame = {}
for line in file_read:
    ## for frame, shape read
    if 'Frame' in line:
        new_line = list(line[:-1])
        frame_name = ''.join(str(letter) for letter in new_line)
        frame[frame_name] = {}
        frame[frame_name]['frame'] = int(frame_name[6:])
        if frame_name in line:
            line_count = file_read.index(line)
    if line == '(416, 416, 3)\n':
        line = list(line[:-1])
        frame[frame_name]['shape'] = ''.join(str(letter) for letter in line)
    if 'Found' in line:
        line = list(line)
        number_end = line.index('b') - 1
        number = line[6:number_end]
        frame[frame_name]['objects_count'] = int(''.join(str(n) for n in number))
        if frame[frame_name]['objects_count'] > 0:
            frame[frame_name]['objects'] = {}
            for i in range(frame[frame_name]['objects_count']):
                obj_line = (line_count + 2) + (i + 1)
                object_dic = frame[frame_name]['objects']['object ' + str(i + 1)] = {}
                object_list = list(file_read[obj_line])

                obj_list = file_read[obj_line]
                sign_name = re.search('(\w+\s)+', obj_list)
                if sign_name != None:
                    sign_name = sign_name.group()[:-1]
                    confidence_score = obj_list[(len(sign_name)+1):(len(sign_name) + 5)]
                    coord_pattern = r'\(([0-9])+\,\s([0-9])+\)'
                    left_top_coord = re.search(r'\(([0-9])+\,\s([0-9])+\)', obj_list).group()
                    left_top_x = re.search(r'([0-9])+', left_top_coord).group()
                    left_top_y = re.search(r'(?<=\,\s)([0-9])+', left_top_coord).group()
                    right_bottom_coord = re.search(
                        r'(?<=\)\s)\(([0-9])+\,\s([0-9])+\)', obj_list).group()
                    right_bottom_x = re.search(r'([0-9])+', right_bottom_coord).group()
                    right_bottom_y = re.search(r'(?<=\,\s)([0-9])+', right_bottom_coord).group()                        
                ## push it all to the object dictionary
                object_dic['class'] = sign_name
                object_dic['confidence_score'] = float(confidence_score)
                object_dic['left_top_coord'] = left_top_coord
                object_dic['right_bottom_coord'] = right_bottom_coord
                object_dic['left_top_x'] = int(left_top_x)
                object_dic['left_top_y'] = int(left_top_y)
                object_dic['right_bottom_x'] = int(right_bottom_x)
                object_dic['right_bottom_y'] = int(right_bottom_y)
            obj_line = (line_count + 2) + (frame[frame_name]['objects_count'] + 1)
            detection_time = float(file_read[obj_line])
            print(obj_line, detection_time)
            frame[frame_name]['detection_time'] = detection_time
        else:
            ## add time to collection
            obj_line = (line_count + 3)
            detection_time = float(file_read[obj_line])
            print('no detection', obj_line, detection_time)
            frame[frame_name]['detection_time'] = detection_time


no detection 3 1.9706918119991315
no detection 7 0.9735546739993879
no detection 11 0.9658381970002665
no detection 15 1.0914586590006365
no detection 19 0.981479070000205
no detection 23 1.067079668999213
no detection 27 1.0615692489991488
no detection 31 1.1094652119991224
no detection 35 1.0403507640003227
no detection 39 1.0496945030008646
no detection 43 1.0768533530008426
no detection 47 1.255485685000167
no detection 51 1.1282681329994375
no detection 55 1.1240170580003905
no detection 59 1.0680767559988453
no detection 63 1.0360380910005915
no detection 67 0.9814900740002486
no detection 71 1.016315957000188
no detection 75 1.0100528080001823
no detection 79 0.9775551180009643
no detection 83 0.9981292209995445
no detection 87 0.9621658449996175
no detection 91 0.9584047889984504
no detection 95 0.977754167999592
no detection 99 0.9936762010001985
no detection 103 0.9912610840001435
no detection 107 1.0034356039996055
no detection 111 0.9956267460001982
116 0.9872128900005919
n

no detection 2762 1.1343460259995481
no detection 2766 1.0552996679998614
no detection 2770 1.0700466170001164
no detection 2774 1.0466675879997638
no detection 2778 1.0479644910010393
no detection 2782 1.0172937579991412
no detection 2786 1.0121045389987557
no detection 2790 0.9999735490000603
no detection 2794 1.0187389469992922
no detection 2798 1.006037278999429
no detection 2802 1.0010892680002144
2807 1.0008919640004024
2812 0.9883592599999247
no detection 2816 0.9953021060009632
no detection 2820 0.9879895230005786
no detection 2824 0.9893081709997205
no detection 2828 1.0066574010015756
no detection 2832 1.013600795999082
no detection 2836 1.055186137000419
no detection 2840 1.001617885000087
no detection 2844 0.9979676570001175
no detection 2848 0.9900628149989643
no detection 2852 1.013308617999428
no detection 2856 1.0726025790008862
no detection 2860 1.016676985000231
no detection 2864 0.9967979869998089
no detection 2868 1.0085027270015416
no detection 2872 1.0767056050008

no detection 4838 0.9970731540015549
no detection 4842 0.991324362999876
no detection 4846 1.019822946000204
no detection 4850 1.0254053849985212
no detection 4854 1.0049124039996968
no detection 4858 1.0148605479989783
no detection 4862 1.009084299001188
no detection 4866 1.0096426789987163
no detection 4870 1.0050765879987011
no detection 4874 0.9969337740003539
no detection 4878 1.020353822999823
no detection 4882 1.005147367999598
no detection 4886 1.0063036319988896
no detection 4890 1.027279506000923
no detection 4894 1.0008737030002521
no detection 4898 1.0098971410006925
no detection 4902 1.0192443710002408
no detection 4906 1.011582096998609
no detection 4910 0.9980127069993614
no detection 4914 1.014269608000177
no detection 4918 1.0062335109996638
no detection 4922 1.0245313449995592
no detection 4926 1.0023610459993506
no detection 4930 0.9809840170000825
no detection 4934 0.9973078219991294
no detection 4938 0.9995930400000361
no detection 4942 0.9876146869992226
no detect

no detection 6939 1.0065344920003554
no detection 6943 0.9987549119996402
no detection 6947 1.0184828040000866
no detection 6951 1.0220705390001967
no detection 6955 1.0113693369985413
no detection 6959 1.0094995540002856
no detection 6963 1.0321428459992603
no detection 6967 1.0169883079997817
no detection 6971 1.002828877999491
no detection 6975 1.0000551619996259
no detection 6979 1.000962404999882
no detection 6983 1.030650278999019
no detection 6987 1.020950302001438
no detection 6991 1.0160657550004544
no detection 6995 1.0401707159999205
no detection 6999 1.0214619710004627
no detection 7003 1.0307484969998768
7008 1.0226038100008736
7013 1.060006241001247
no detection 7017 1.0423626100000547
no detection 7021 1.0505477599999722
no detection 7025 1.0074704639991978
no detection 7029 1.0230569049999758
no detection 7033 1.038008268998965
no detection 7037 1.0345231229985075
no detection 7041 1.0362224420005077
no detection 7045 0.9948147260001861
no detection 7049 1.0481034059994

no detection 9247 0.993480991000979
no detection 9251 1.0121547909984656
no detection 9255 0.9994740920010372
no detection 9259 1.003099393001321
no detection 9263 0.9844267170010426
no detection 9267 0.9924482570004329
no detection 9271 0.9893430070005707
no detection 9275 0.9849787900002411
no detection 9279 0.9976844390002952
no detection 9283 0.9940675369998644
no detection 9287 1.0015367129999504
no detection 9291 0.990680605000307
9296 1.0103862979995029
9301 1.0213930429999891
9306 1.008565293999709
no detection 9310 0.9975405380009761
no detection 9314 0.997573516000557
no detection 9318 0.99911779499962
no detection 9322 0.9948168669998267
no detection 9326 0.9971060280004167
no detection 9330 1.0109047679998184
no detection 9334 1.0170489569991332
no detection 9338 1.0144511920007062
no detection 9342 1.0092890449996048
no detection 9346 0.9980876499994338
no detection 9350 1.0161520340006973
no detection 9354 1.0126789179994375
no detection 9358 1.011798176999946
9363 1.0047

no detection 10709 1.0095421389996773
no detection 10713 1.0079883050002536
no detection 10717 1.0176598159996502
no detection 10721 1.0062217790000432
no detection 10725 1.006365168001139
no detection 10729 1.0139421339990804
10734 0.9999429950003105
no detection 10738 1.00152632299978
no detection 10742 0.9976128239995887
no detection 10746 1.007836426000722
no detection 10750 1.0083343969999987
no detection 10754 0.9914761400013958
no detection 10758 1.0093559650013049
no detection 10762 1.0092553329996008
no detection 10766 1.0027504729987413
no detection 10770 1.023442845000318
no detection 10774 1.01748446000056
no detection 10778 1.020221691000188
no detection 10782 1.0097325279984943
no detection 10786 1.0264418769984331
no detection 10790 1.0157941540001048
no detection 10794 1.0175074269991455
no detection 10798 1.0186554779993457
no detection 10802 1.0157322680006473
no detection 10806 1.0142976980005187
no detection 10810 1.003234389998397
no detection 10814 1.0009231050007

In [4]:
frame['Frame 2']

{'frame': 2,
 'shape': '(416, 416, 3)',
 'objects_count': 0,
 'detection_time': 0.9735546739993879}

In [5]:
frame['Frame 1688']

{'frame': 1688,
 'shape': '(416, 416, 3)',
 'objects_count': 4,
 'objects': {'object 1': {'class': 'keep left sign',
   'confidence_score': 0.66,
   'left_top_coord': '(1061, 302)',
   'right_bottom_coord': '(1108, 395)',
   'left_top_x': 1061,
   'left_top_y': 302,
   'right_bottom_x': 1108,
   'right_bottom_y': 395},
  'object 2': {'class': 'speed sign',
   'confidence_score': 0.43,
   'left_top_coord': '(962, 0)',
   'right_bottom_coord': '(1141, 106)',
   'left_top_x': 962,
   'left_top_y': 0,
   'right_bottom_x': 1141,
   'right_bottom_y': 106},
  'object 3': {'class': 'speed sign',
   'confidence_score': 0.58,
   'left_top_coord': '(503, 126)',
   'right_bottom_coord': '(550, 168)',
   'left_top_x': 503,
   'left_top_y': 126,
   'right_bottom_x': 550,
   'right_bottom_y': 168},
  'object 4': {'class': 'speed sign',
   'confidence_score': 0.78,
   'left_top_coord': '(1089, 141)',
   'right_bottom_coord': '(1128, 191)',
   'left_top_x': 1089,
   'left_top_y': 141,
   'right_bottom_

In [6]:
print(line_count)

11779


In [7]:
for k, v in frame.items():
    print(k, v)

Frame 1 {'frame': 1, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.9706918119991315}
Frame 2 {'frame': 2, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9735546739993879}
Frame 3 {'frame': 3, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9658381970002665}
Frame 4 {'frame': 4, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0914586590006365}
Frame 5 {'frame': 5, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.981479070000205}
Frame 6 {'frame': 6, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.067079668999213}
Frame 7 {'frame': 7, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0615692489991488}
Frame 8 {'frame': 8, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.1094652119991224}
Frame 9 {'frame': 9, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0403507640003227}
Frame 10 {'frame': 10, 'shape': '(416, 416, 3)', 'objects

Frame 318 {'frame': 318, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0170619010004884}
Frame 319 {'frame': 319, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9972032439982286}
Frame 320 {'frame': 320, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9895297260009102}
Frame 321 {'frame': 321, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9806685829989874}
Frame 322 {'frame': 322, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9978469329998916}
Frame 323 {'frame': 323, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0163101750003989}
Frame 324 {'frame': 324, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0116614310009027}
Frame 325 {'frame': 325, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9790575689985417}
Frame 326 {'frame': 326, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9792743500001961}
Frame 327 {'frame':

Frame 841 {'frame': 841, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0551840700009052}
Frame 842 {'frame': 842, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0249130279989913}
Frame 843 {'frame': 843, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0021259960012685}
Frame 844 {'frame': 844, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0144794980005827}
Frame 845 {'frame': 845, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0173018650002632}
Frame 846 {'frame': 846, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0606650849986181}
Frame 847 {'frame': 847, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.022932400001082}
Frame 848 {'frame': 848, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0016614150008536}
Frame 849 {'frame': 849, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0067597639990709}
Frame 850 {'frame': 

Frame 1091 {'frame': 1091, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0807832440004859}
Frame 1092 {'frame': 1092, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0191444890006096}
Frame 1093 {'frame': 1093, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0014131129992165}
Frame 1094 {'frame': 1094, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0255031310007325}
Frame 1095 {'frame': 1095, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0121176169996033}
Frame 1096 {'frame': 1096, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0010192849986197}
Frame 1097 {'frame': 1097, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0135763590005809}
Frame 1098 {'frame': 1098, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0024875500002963}
Frame 1099 {'frame': 1099, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9868817480000871}
F

Frame 1840 {'frame': 1840, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.02593051400072}
Frame 1841 {'frame': 1841, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0246661240016692}
Frame 1842 {'frame': 1842, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0291717359996255}
Frame 1843 {'frame': 1843, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0232736530015245}
Frame 1844 {'frame': 1844, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.007423643999573}
Frame 1845 {'frame': 1845, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0028325359999144}
Frame 1846 {'frame': 1846, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0063215000009222}
Frame 1847 {'frame': 1847, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.000959196999247}
Frame 1848 {'frame': 1848, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9925446609995561}
Frame

Frame 2090 {'frame': 2090, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'keep left sign', 'confidence_score': 0.84, 'left_top_coord': '(1151, 302)', 'right_bottom_coord': '(1187, 345)', 'left_top_x': 1151, 'left_top_y': 302, 'right_bottom_x': 1187, 'right_bottom_y': 345}}, 'detection_time': 1.0092176090001885}
Frame 2091 {'frame': 2091, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0124595150009554}
Frame 2092 {'frame': 2092, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.021774816999823}
Frame 2093 {'frame': 2093, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0103569789989706}
Frame 2094 {'frame': 2094, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0046577419998357}
Frame 2095 {'frame': 2095, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0306567719999293}
Frame 2096 {'frame': 2096, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.057785

Frame 2840 {'frame': 2840, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0052003800010425}
Frame 2841 {'frame': 2841, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9963415350011928}
Frame 2842 {'frame': 2842, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0002339139991818}
Frame 2843 {'frame': 2843, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9971778369999811}
Frame 2844 {'frame': 2844, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.006279489998633}
Frame 2845 {'frame': 2845, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9972178589996474}
Frame 2846 {'frame': 2846, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9984170410007209}
Frame 2847 {'frame': 2847, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9941344119997666}
Frame 2848 {'frame': 2848, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0015213110000332}
Fr

In [8]:
type(frame['Frame 2596']['objects']['object 1']['confidence_score'])

float

In [9]:
## now to address the chain of events for counting
## think about the flow of data
## read the flow of counts into a list
## as the list enters, we only want to keep the chain of 10 frames
frames_list_found = []
found_plus_next_ten = []
for k, v in frame.items():
    i = 0
    if v['objects_count'] > 0:
        for j in range(v['objects_count']):
            i += 1
            obj = v['objects']['object ' + str(i)]
            frames_list_found += [frame[k]['frame'], obj['confidence_score']]
            ten_frames = []
            for l in range(10):
                obj_mod = frame['Frame ' + str(l + frame[k]['frame'])]
                print(obj_mod)
                ten_frames.append(obj_mod)
#                 print('this', obj_mod)
#                 if obj_mod['objects_count'] > 0:
#                     for m in range(obj_mod['objects_count']):
#                         obj_mod = obj_mod['objects']['object ' + str(i)]
#                         print('obj_mod?', obj_mod)
#                         ten_frames.append([[obj_mod['frame'], obj_mod['confidence_score']]])
#                         print(l + obj_mod['frame'])
            print('end\n')
            found_plus_next_ten.append(ten_frames)
#             print(found_plus_next_ten)

{'frame': 29, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'no entry sign', 'confidence_score': 0.3, 'left_top_coord': '(1199, 39)', 'right_bottom_coord': '(1341, 178)', 'left_top_x': 1199, 'left_top_y': 39, 'right_bottom_x': 1341, 'right_bottom_y': 178}}, 'detection_time': 0.9872128900005919}
{'frame': 30, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.970679347999976}
{'frame': 31, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9735048559996358}
{'frame': 32, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'no entry sign', 'confidence_score': 0.43, 'left_top_coord': '(549, 69)', 'right_bottom_coord': '(610, 152)', 'left_top_x': 549, 'left_top_y': 69, 'right_bottom_x': 610, 'right_bottom_y': 152}}, 'detection_time': 0.9756631119998929}
{'frame': 33, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9652219529998547}
{'frame': 34, 'shape': '(416, 416, 3)', 'objects_c

{'frame': 515, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'speed sign', 'confidence_score': 0.76, 'left_top_coord': '(101, 35)', 'right_bottom_coord': '(218, 118)', 'left_top_x': 101, 'left_top_y': 35, 'right_bottom_x': 218, 'right_bottom_y': 118}}, 'detection_time': 1.0537800040001457}
{'frame': 516, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0707207030009158}
{'frame': 517, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0389489170011075}
{'frame': 518, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0508189400006813}
{'frame': 519, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0611922180014517}
{'frame': 520, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.046245158999227}
{'frame': 521, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0844574160000775}
{'frame': 522, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0

{'frame': 1946, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9999448220005434}
{'frame': 1947, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0078459809992637}
{'frame': 1948, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0357945289997588}
{'frame': 1949, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0358299859999534}
{'frame': 1950, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.021056146999399}
{'frame': 1951, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0399628030008898}
end

{'frame': 1958, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'no entry sign', 'confidence_score': 0.39, 'left_top_coord': '(421, 141)', 'right_bottom_coord': '(440, 173)', 'left_top_x': 421, 'left_top_y': 141, 'right_bottom_x': 440, 'right_bottom_y': 173}}, 'detection_time': 1.0000432979995821}
{'frame': 1959, 'shape': '(416, 416, 3)', 'objects_count': 0, 'de

{'frame': 2096, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 1.0577858969991212}
end

{'frame': 2088, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'keep left sign', 'confidence_score': 0.44, 'left_top_coord': '(950, 277)', 'right_bottom_coord': '(968, 304)', 'left_top_x': 950, 'left_top_y': 277, 'right_bottom_x': 968, 'right_bottom_y': 304}}, 'detection_time': 1.0059101860006194}
{'frame': 2089, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'keep left sign', 'confidence_score': 0.69, 'left_top_coord': '(1030, 278)', 'right_bottom_coord': '(1058, 314)', 'left_top_x': 1030, 'left_top_y': 278, 'right_bottom_x': 1058, 'right_bottom_y': 314}}, 'detection_time': 1.0101553439999407}
{'frame': 2090, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'keep left sign', 'confidence_score': 0.84, 'left_top_coord': '(1151, 302)', 'right_bottom_coord': '(1187, 345)', 'left_top_x': 115

In [10]:
print(len(found_plus_next_ten))
count = 0
for k, v in frame.items():
    if frame[k]['objects_count'] > 0:
        count += frame[k]['objects_count']
print(count)
for i in range(len(found_plus_next_ten[0])):
    print(found_plus_next_ten[0][i])

359
359
{'frame': 29, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'no entry sign', 'confidence_score': 0.3, 'left_top_coord': '(1199, 39)', 'right_bottom_coord': '(1341, 178)', 'left_top_x': 1199, 'left_top_y': 39, 'right_bottom_x': 1341, 'right_bottom_y': 178}}, 'detection_time': 0.9872128900005919}
{'frame': 30, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.970679347999976}
{'frame': 31, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9735048559996358}
{'frame': 32, 'shape': '(416, 416, 3)', 'objects_count': 1, 'objects': {'object 1': {'class': 'no entry sign', 'confidence_score': 0.43, 'left_top_coord': '(549, 69)', 'right_bottom_coord': '(610, 152)', 'left_top_x': 549, 'left_top_y': 69, 'right_bottom_x': 610, 'right_bottom_y': 152}}, 'detection_time': 0.9756631119998929}
{'frame': 33, 'shape': '(416, 416, 3)', 'objects_count': 0, 'detection_time': 0.9652219529998547}
{'frame': 34, 'shape': '(416, 416, 3)', 'o

# Do we want to count the difference in frequency between objects detected, via summary statistics?
Yes (frequency of 1 object being detected, frequency of multiple objects detected)

In [11]:
def intersection_over_union(boxA, boxB):
	"""left, top, right bottom"""
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	#compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	#compute the area of both the prediction and ground truth
	boxA_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	#compute the intersection over union by taking the intersection
	#area and dividing it by the sum of prediction + ground-truth
	#areas - the intersection area
	iou = interArea / float(boxA_area + boxB_area - interArea)
	# return the intersection over union value
	return iou

def larger_intersection_over_union(boxA, boxB):
	"""
    left, top, right bottom, amending so that it becomes an imaginary box in nature
    possibly required for right side, or turning where the extremes to the left or right should be larger
    IOU?
    
    It might be incorrect.
    """
	amended_boxA = list(boxA)
	amended_boxA[0] = amended_boxA[0] - 200
	amended_boxA[2] = amended_boxA[2] + 200
	xA = max(amended_boxA[0], boxB[0])
	yA = max(amended_boxA[1], boxB[1])
	xB = min(amended_boxA[2], boxB[2])
	yB = min(amended_boxA[3], boxB[3])
	#compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	#compute the area of both the prediction and ground truth
	boxA_area = (
		amended_boxA[2] - amended_boxA[0] + 1) * (amended_boxA[3] - amended_boxA[1] + 1)
	boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	#compute the intersection over union by taking the intersection
	#area and dividing it by the sum of prediction + ground-truth
	#areas - the intersection area
	iou = interArea / float(boxA_area + boxB_area - interArea)
	# return the intersection over union value
	return iou

In [24]:
## do they need to appear in sequence?
## assume that they do for now
## also how many objects in each item?
## need to ensure that what is passed in


## Notes for LL
## Approach is to identify where an object is identified, push that frame and the next 9 into a list
## then go through the list, and commpare each frame to the next where the object counts follow one another
## then for each object compare each object with each other object until all detected objects have an IOU calculated
## if the IOU is not zero, and the classes match, print the output.


## Is the general approach correct?

## steps that need to be done.
## take into account the confidence score? of the item itself
## where it's true that each frame supercedes a previous frame to drop the first frame
## only when it's true that the frames were in order.

##total count IOU is too simple
total_count_IOU = 0
intersection_boxes_criteria_met = []
# not used yet
intersection_boxes_not_met = []
intersection_boxes_met_not_confidence = []
for i in range(len(found_plus_next_ten)):
    for j in range(len(found_plus_next_ten[i]) - 1):
        boxA = found_plus_next_ten[i][j]
        boxB = found_plus_next_ten[i][j + 1]
        if boxA['objects_count'] > 0 and boxB['objects_count'] > 0:
            for k in range(boxA['objects_count']):
                for l in range(boxB['objects_count']):
                    new_boxA = [
                        boxA['objects']['object ' + str(k+1)]['left_top_x'],
                        boxA['objects']['object ' + str(k+1)]['left_top_y'],
                        boxA['objects']['object ' + str(k+1)]['right_bottom_x'],
                        boxA['objects']['object ' + str(k+1)]['right_bottom_y'],
                    ]
                    box_a_class = boxA['objects']['object ' + str(k+1)]['class']
                    box_a_confidence = boxA['objects']['object ' + str(k+1)]['confidence_score']
                    new_boxB = [
                        boxB['objects']['object ' + str(l + 1)]['left_top_x'],
                        boxB['objects']['object ' + str(l + 1)]['left_top_y'],
                        boxB['objects']['object ' + str(l + 1)]['right_bottom_x'],
                        boxB['objects']['object ' + str(l + 1)]['right_bottom_y']
                    ]
                    box_b_class = boxB['objects']['object ' + str(l + 1)]['class']
                    box_b_confidence = boxB['objects']['object ' + str(l+1)]['confidence_score']
                    if(intersection_over_union(new_boxA, new_boxB) != 0.0) \
                            and box_a_class == box_b_class \
                            and box_a_confidence > 0.60 \
                            and box_b_confidence > 0.60:
#                         print(new_boxA, '\n', new_boxB)
#                         print(intersection_over_union(new_boxA, new_boxB))
                        total_count_IOU += 1
                        intersection_boxes_criteria_met += [intersection_over_union(new_boxA, new_boxB)]
                    else:
                        if intersection_over_union(new_boxA, new_boxB) != 0.0:
                            intersection_boxes_met_not_confidence += [intersection_over_union(new_boxA, new_boxB)]
                        else:
                            intersection_boxes_not_met += [intersection]
                        ## super lazy way of doing this
#                         boxA = found_plus_next_ten[i][k]
#                         boxB = found_plus_next_ten[i][l]
#                         #this bit is now incorrect.
#                         print('boxA object', k, boxA['frame'], 'boxB object', l, boxB['frame'])
#                         print('boxA confidence', boxA['objects']['object 1']['confidence_score'], '\n')
#                         print('boxB confidence', boxB['objects']['object 1']['confidence_score'], '\n')

## it's currently counting the same IOU's, so if I include the frames then drop them that should work.
print('IOU count occurences', total_count_IOU)
print('IOU conditions met list', len(intersection_boxes_criteria_met))
# print('IOU conditions not met', len(intersection_boxes_criteria_not_met))
print('IOU condition met, but confidence not', len(intersection_boxes_met_not_confidence))

IOU count occurences 55
IOU conditions met list 55
IOU condition met, but confidence not 254


In [13]:
print(found_plus_next_ten[233][0])
print(found_plus_next_ten[233][1])
a = found_plus_next_ten[233][0]
b = found_plus_next_ten[233][1]
for i in range(a['objects_count']):
    for j in range(b['objects_count']):
        boxA = a['objects']['object ' + str(i+1)]['left_top_x']
        boxB = b['objects']['object ' + str(j+1)]['left_top_x']
        print(boxA, boxB, i+1 , j+1)

{'frame': 2113, 'shape': '(416, 416, 3)', 'objects_count': 2, 'objects': {'object 1': {'class': 'no entry sign', 'confidence_score': 0.46, 'left_top_coord': '(781, 118)', 'right_bottom_coord': '(866, 194)', 'left_top_x': 781, 'left_top_y': 118, 'right_bottom_x': 866, 'right_bottom_y': 194}, 'object 2': {'class': 'keep left sign', 'confidence_score': 0.55, 'left_top_coord': '(822, 290)', 'right_bottom_coord': '(848, 319)', 'left_top_x': 822, 'left_top_y': 290, 'right_bottom_x': 848, 'right_bottom_y': 319}}, 'detection_time': 1.011015000000043}
{'frame': 2114, 'shape': '(416, 416, 3)', 'objects_count': 3, 'objects': {'object 1': {'class': 'no entry sign', 'confidence_score': 0.36, 'left_top_coord': '(881, 62)', 'right_bottom_coord': '(955, 214)', 'left_top_x': 881, 'left_top_y': 62, 'right_bottom_x': 955, 'right_bottom_y': 214}, 'object 2': {'class': 'keep left sign', 'confidence_score': 0.74, 'left_top_coord': '(919, 304)', 'right_bottom_coord': '(959, 348)', 'left_top_x': 919, 'left_to

In [14]:
print(intersection_over_union([153, 79, 225, 172], [54,43,165,155]))

0.0540584327914889


1